# Exploration 4. Text Generation  연극 대사 문장 생성



순환신경망(RNN) 을 이용한 언어 모델로 연극의 대사를 학습해서 스스로 연극 대사 문장을 생성해내는 인공지능을 만들어보자.

__셰익스피어와 버금가는 극작가 언어 모델__ 을 기대하며.

![image](https://user-images.githubusercontent.com/84179578/126730385-cba700d8-37dc-4fd5-845a-9f81b7164719.png)

## 1. 데이터 준비

### 데이터 불러오기

In [1]:
import os, re
import numpy as np
import tensorflow as tf

file_path = 'EP04_data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


데이터의 형태를 보면 아래와 같이 되어있다.
![image](https://user-images.githubusercontent.com/84179578/126731288-a3c6b6e5-a98e-4a47-b763-aaee5cf9c0a1.png)

여기서 필요한 문장은 대사인 부분이므로,  화자가 표기된 문장과 공백인 문장은 제외시킨다.   

아래와 같이 제외시킬 문장의 특징을 찾아 문장을 제외시킨다.
- 화자가 표기된 문장 : `:`로 끝나는 문장
- 공백인 문장 : 문장의 길이를 검사하여 길이가 0 인 문장

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장 건너뜀.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장 건너뜀.

    if idx > 9: break   # 일단 문장 10개만 확인
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


### 토큰화 (Tokenize)
다음과 같은 순서로 문장을 일정한 기준으로 쪼개는 __토큰화 (Tokenize)__ 를 진행한다.

_정규표현식을 이용한 필터링이 유용하게 사용됨_   -> 정규표현식에 대한 자세한 내용은 아래 블로그 링크 참조
> [AIFFEL FD #05 문자열, 파일 다루기 - 정규 표현식 (Regular expression, regex)](https://velog.io/@leejaejun/AIFFEL-FD-05-%EB%AC%B8%EC%9E%90%EC%97%B4-%ED%8C%8C%EC%9D%BC-%EB%8B%A4%EB%A3%A8%EA%B8%B0)

- 1. 소문자로 바꾸고, 양쪽 공백을 지움
- 2. 특수문자 양쪽에 공백을 넣음
- 3. 여러개의 공백은 하나의 공백으로 바꿈
- 4. `a-zA-Z?.!,¿`가 아닌 모든 문자를 하나의 공백으로 바꿈
- 5. 다시 양쪽 공백을 지움
- 6. 문장 시작에는 `<start>`, 끝에는 `<end>`를 추가

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                     # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)     # 2
    sentence = re.sub(r'[" "]+', " ", sentence)             # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)   # 4
    sentence = sentence.strip()                             # 5
    sentence = '<start> ' + sentence + ' <end>'             # 6
    return sentence

# 샘플 문장 으로 잘되는지 테스트
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용한다.  

- `tf.keras.preprocessing.text.Tokenizer` 설명
  - `num_words` : 7000단어를 기억할 수 있는 tokenizer 생성
  - `filters` : 이전 단계에서 이미 문장을 정제했으니 filters가 필요하지않음
  - `oov_token`: 7000단어에 포함되지 못한 단어는 `<unk>`로 변환
 
  
- `tf.keras.preprocessing.sequence.pad_sequences` 설명
  - 입력 데이터의 시퀀스 길이를 일정하게 맞춰줌
  - 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춤
  - 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용



In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x0000022CA884F8C8>


생성된 텐서 데이터를 5번째행, 10 번째 열까지만 출력해서 형태를 보자

In [6]:
print(tensor[:5, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]
 [   2 1201    5 1201    5    3    0    0    0    0]
 [   2  199    4   11   92 1021  298   18 2314  513]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


텐서 데이터는 모두 정수로 이루어져있다. 또한 모두 2번 인덱스로 시작하는데 2번 인덱스라 `<start>` 이기 때문인 것을 쉽게 알 수 있다.

In [8]:
src_input = tensor[:, :-1] # 소스문장 생성 

tgt_input = tensor[:, 1:]  # 타켓 문장 생성

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성한다. 마지막 토큰은`<end>`가 아니라 `<pad>`일 가능성이 높습니다.

tensor에서 `<start>`를 잘라내서 타겟 문장을 생성한다.

### 데이터셋 객체 생성

In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋 생성
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

## 모델 학습

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.

for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣음
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 2.4839316e-04,  1.2015342e-04, -3.5721785e-04, ...,
          2.8301118e-04,  1.4798935e-04, -3.4065481e-04],
        [ 4.2692857e-04,  1.7370727e-04, -4.0230996e-04, ...,
          3.1553474e-04,  2.6398891e-04, -1.9628822e-04],
        [ 3.2791414e-04,  4.4448982e-04, -4.9103971e-04, ...,
          4.9881014e-04,  2.8514100e-04, -2.8807076e-04],
        ...,
        [-6.1485905e-04,  1.7636900e-03, -2.8105821e-03, ...,
         -1.6348425e-03, -2.4537423e-03,  1.6620897e-03],
        [-5.3172838e-04,  2.0138898e-03, -2.9725763e-03, ...,
         -2.1892895e-03, -2.5777128e-03,  1.9230701e-03],
        [-4.4037620e-04,  2.2203559e-03, -3.0923714e-03, ...,
         -2.6918913e-03, -2.6764076e-03,  2.1633166e-03]],

       [[ 2.4839316e-04,  1.2015342e-04, -3.5721785e-04, ...,
          2.8301118e-04,  1.4798935e-04, -3.4065481e-04],
        [ 4.4742698e-04,  1.1057055e-04, -5.4323493e-04, ...,
          1.8108700e-04, -

모델의 최종 출력 텐서 shape 을 보면 `shape=(256,20,7001)` 임을 확인 할 수 잇다.
각 숫자의 이미는 아래와 같다.
- `256` : 이전 스텝에서 지정한 배치 사이즈  

- `20` : `tf.keras.layers.LSTM(hidden_size, return_sequences=True)` 로 호출한 LSTM 레이어에서 `return_sequences=True`이라고 지정한 부분에 있습니다. 즉, LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력한다는 의미입니다. 만약 `return_sequences=False`였다면 LSTM 레이어는 1개의 벡터만 출력했을 것입니다.  

- `7001` : Dense 레이어의 출력 차원수. 7001개의 단어 중 어느 단어의 확률이 가장 높을지를 모델링

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [13]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> he")

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i")

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> she")

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> If")